# Emojify Deep Learning FER-2013

Source : [Data-Flair](https://data-flair.training/blogs/create-emoji-with-deep-learning/)

## Import modules

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten
from keras.layers import MaxPooling2D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D

import numpy as np
import cv2

## Initialize training and validation generators

In [7]:
train_dir = 'data/train'
val_dir = 'data/test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                  train_dir,
                  target_size=(48, 48),
                  batch_size=64,
                  color_mode='grayscale',
                  class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
                       val_dir,
                       target_size=(48, 48),
                       batch_size=64,
                       color_mode='grayscale',
                       class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


## Build convolution neural network architecture

In [8]:
emotion_model = Sequential()

################# LAYER 1 ###########################
emotion_model.add(Conv2D(32, kernel_size=(3, 3), 
                         activation='relu', 
                         input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3),
                         activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

################# LAYER 2 ###########################
emotion_model.add(Conv2D(128, kernel_size=(3, 3),
                         activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3),
                         activation='relu'))
emotion_model.add(Dropout(0.25))

################# LAYER 3 ###########################
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

## Compile and train the model

In [12]:
emotion_model.compile(loss='categorical_crossentropy', 
                      optimizer=Adam(lr=0.0001, decay=1e-6), 
                      metrics=['accuracy'])

emotion_model_info = emotion_model.fit(
                     train_generator,
                     steps_per_epoch=28709 // 64,
                     epochs=50,
                     validation_data=validation_generator,
                     validation_steps=7178//64)

Epoch 1/50
448/448 [==============================] - 892s 2s/step - loss: 1.8083 - accuracy: 0.2610 - val_loss: 1.6141 - val_accuracy: 0.3811
Epoch 2/50
448/448 [==============================] - 227s 506ms/step - loss: 1.6047 - accuracy: 0.3816 - val_loss: 1.5106 - val_accuracy: 0.4224
Epoch 3/50
448/448 [==============================] - 250s 558ms/step - loss: 1.4996 - accuracy: 0.4273 - val_loss: 1.4279 - val_accuracy: 0.4555
Epoch 4/50
448/448 [==============================] - 249s 555ms/step - loss: 1.4246 - accuracy: 0.4578 - val_loss: 1.3722 - val_accuracy: 0.4750
Epoch 5/50
448/448 [==============================] - 256s 570ms/step - loss: 1.3442 - accuracy: 0.4914 - val_loss: 1.3238 - val_accuracy: 0.4939
Epoch 6/50
448/448 [==============================] - 240s 536ms/step - loss: 1.2949 - accuracy: 0.5083 - val_loss: 1.2942 - val_accuracy: 0.5049
Epoch 7/50
448/448 [==============================] - 247s 551ms/step - loss: 1.2372 - accuracy: 0.5349 - val_loss: 1.2482 - va

## Saving the model

In [13]:
emotion_model.save('emotion_model.h5')

In [14]:
emotion_model.save_weights('model.h5')

## Load model

In [2]:
from keras.models import load_model

emotion_model = load_model('emotion_model.h5')

## Integrating opencv

In [3]:
cv2.ocl.setUseOpenCL(False)

emotion_dict = {0 : "Angry",
                1 : "Disgusted",
                2 : "Fearful",
                3 : "Happy",
                4 : "Neutral",
                5 : "Sad",
                6 : "Surprised"}


cap = cv2.VideoCapture(0)

while True:
  ret, frame = cap.read()
  if not ret:
    break
  bounding_box = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
  gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  num_faces = bounding_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

  for (x, y, w, h) in num_faces:
    cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
    roi_gray_frame = gray_frame[y:y+h, x:x+w]
    cropped_image = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
    emotion_prediction = emotion_model.predict(cropped_img)
    maxindex = int(np.argmax(emotion_prediction))
    cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    cv2.imshow("Video", cv2.resize(frame, (1200, 860), interpolation = cv2.INTER_CUBIC))

    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
    
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-1i5nllza\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'
